In [ ]:
import pickle as pkl    
import os
from pcdet.ops.iou3d_nms import iou3d_nms_utils
import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

import torch
os.chdir('/home/cgriesbacher/thesis/3DTrans/')

In [63]:
def calc_ious(pseudo_labels, gt_labels):
    #add iou score to the dataframe, default is 0
    pseudo_labels = pseudo_labels.assign(iou = 0)
    pseudo_labels = pseudo_labels.assign(true_label = "Background")
    pseudo_labels = pseudo_labels.assign(gt_id = -1)
    gt_labels = gt_labels.assign(iou = 0)

    #det detection frame ids
    ps_frame_ids = pseudo_labels['frame_id'].unique()

    # Loop over all frames
    for ps_frame_id in tqdm.tqdm(ps_frame_ids):

        gt_labels_frame = gt_labels[gt_labels['frame_id'] == ps_frame_id]
        pseudo_labels_frame = pseudo_labels[pseudo_labels['frame_id'] == ps_frame_id]

        if len(gt_labels_frame) == 0 or len(pseudo_labels_frame) == 0:
            #print(f"Frame {ps_frame_id} has no gts or no detections")
            continue

        # Convert the boxes to numpy
        gt_boxes_frame = np.stack(gt_labels_frame["gt_boxes_lidar"].values)
        ps_boxes_frame = np.stack(pseudo_labels_frame["boxes_lidar"].values)

        #iou_matrix = iou3d_nms_utils.boxes_iou3d_gpu(torch.tensor(gt_boxes_frame, dtype=torch.float, device='cuda'), torch.tensor(ps_boxes_frame, dtype=torch.float, device='cuda')).cpu().numpy()
        iou_matrix = iou3d_nms_utils.boxes_iou_bev(torch.tensor(gt_boxes_frame, dtype=torch.float, device='cuda'), torch.tensor(ps_boxes_frame, dtype=torch.float, device='cuda')).cpu().numpy()
        #iou_matrix = iou3d_nms_utils.boxes_bev_iou_cpu(gt_boxes_frame, ps_boxes_frame)
        
        # Get the max iou for each det box
        max_ious_ps = np.max(iou_matrix, axis=0)
        max_ious_ps_idx = np.argmax(iou_matrix, axis=0)

        detection_mask = max_ious_ps > 0
        max_ious_ps = max_ious_ps[detection_mask]
        max_ious_ps_idx = max_ious_ps_idx[detection_mask]
        pseudo_labels_frame_masked = pseudo_labels_frame[detection_mask]

        # get class for each detection
        cls_dets = gt_labels_frame['names'].values[max_ious_ps_idx]
        gt_id_dets = gt_labels_frame['gt_id'].values[max_ious_ps_idx]

        
        # Update the iou score and gt_id for each detection in the original DataFrame
        pseudo_labels.loc[pseudo_labels_frame_masked.index, 'iou'] = max_ious_ps
        pseudo_labels.loc[pseudo_labels_frame_masked.index, 'gt_id'] = gt_id_dets

        pseudo_labels.loc[pseudo_labels_frame_masked.index, 'true_label'] = cls_dets
        # print(f"Percentage of detection ious greater than 0: {round(len(max_ious_ps[max_ious_ps > 0])/len(max_ious_ps),2)}")

        # Get max_iou for each gt box
        max_ious_gts = np.max(iou_matrix, axis=1)

        # Update the iou score for each gt in the original DataFrame
        gt_labels.loc[gt_labels_frame.index, 'iou'] = max_ious_gts
        #percentage of ious that are greater than 0
        # print(f"Percentage of gt ious greater than 0: {round(len(max_ious_gts[max_ious_gts > 0])/len(max_ious_gts),2)}")

        # XXX: only update if score is higher than current score


    iou_thresholds = [0.1, 0.3, 0.5, 0.7]

    #for each detection, add detected_0.3, detected_0.5, detected_0.7
    for iou_threshold in iou_thresholds:
        pseudo_labels[f'detected_{iou_threshold}'] = pseudo_labels['iou'] >= iou_threshold
        gt_labels[f'detected_{iou_threshold}'] = gt_labels['iou'] >= iou_threshold

    return pseudo_labels, gt_labels


def load_detections(path, df, train_dataset=None, eval_dataset=None):
    with open(path, 'rb') as f:
        dets = pkl.load(f)

    #make a dataframe
    df_dets_dataset = pd.DataFrame(dets)
    
    #get datasset name from path
    if eval_dataset == None:
        eval_dataset = path.split('/')[-2]
    if train_dataset == None:
        train_dataset = path.split('/')[-8].split('_')[0]

    #add a column with the dataset name
    df_dets_dataset['eval_dataset'] = eval_dataset
    df_dets_dataset['train_dataset'] = train_dataset

    cols_to_explode = ['name', 'score', 'boxes_lidar', 'pred_labels', 'train_dataset', 'eval_dataset', 'frame_id']
    exploded_list = [df_dets_dataset[col].explode() for col in cols_to_explode]
    exploded_df = pd.concat(exploded_list, axis=1)

    # Adding non-exploded columns
    for col in df_dets_dataset.columns:
        if col not in cols_to_explode:
            exploded_df[col] = df[col].values[0]

    # Reset index if needed
    df_det_expl = exploded_df.reset_index(drop=True)

    df_det_expl = df_det_expl.dropna()

    df_new = pd.concat([df, df_det_expl], axis=0)
    df_new = df_new.reset_index(drop=True)

    print("loaded {} detections on dataset {} with model trained on {}".format(len(df_det_expl), eval_dataset, train_dataset))

    return df_new.copy()


def load_gt_labels(split):

    assert split in ['train', 'val']
    dataset_names = ["avltruck", "avlrooftop", "zod"]

    #annotation paths
    annotation_path_avltruck = "/home/cgriesbacher/thesis/3DTrans/data/avltruck/avl_infos_"+split+".pkl"
    annotation_path_avlrooftop = "/home/cgriesbacher/thesis/3DTrans/data/avlrooftop/avl_infos_"+split+".pkl"
    annotation_path_zod = "/home/cgriesbacher/thesis/3DTrans/data/zod/zod_infos_"+split+"_full.pkl"

    #load the annotations and make a dataframe
    with open(annotation_path_avltruck, 'rb') as f:
        annotations_avltruck = pkl.load(f)
    with open(annotation_path_avlrooftop, 'rb') as f:
        annotations_avlrooftop = pkl.load(f)
    with open(annotation_path_zod, 'rb') as f:
        annotations_zod = pkl.load(f)

    avltruck_content = []
    for i in range(len(annotations_avltruck)):
        content = {}
        content["names"] = annotations_avltruck[i]["annos"]['name']
        content["gt_boxes_lidar"] = annotations_avltruck[i]["annos"]['gt_boxes_lidar']
        content['dataset'] = dataset_names[0]
        content['frame_id'] = annotations_avltruck[i]["point_cloud"]['lidar_idx']
        avltruck_content.append(content)

    avlrooftop_content = []
    for i in range(len(annotations_avlrooftop)):
        content = {}
        content["names"] = annotations_avlrooftop[i]["annos"]['name']
        content["gt_boxes_lidar"] = annotations_avlrooftop[i]["annos"]['gt_boxes_lidar']
        content['dataset'] = dataset_names[1]
        content['frame_id'] = annotations_avlrooftop[i]["point_cloud"]['lidar_idx']
        avlrooftop_content.append(content)

    zod_content = []
    for i in range(len(annotations_zod)):
        content = {}
        content["names"] = annotations_zod[i]["annos"]['name']
        content["gt_boxes_lidar"] = annotations_zod[i]["annos"]['gt_boxes_lidar']
        content['dataset'] = dataset_names[2]
        content['frame_id'] = annotations_zod[i]["point_cloud"]['lidar_idx']
        zod_content.append(content)

    #make a dataframe
    df_avltruck_annos = pd.DataFrame(avltruck_content)
    df_avlrooftop_annos = pd.DataFrame(avlrooftop_content)
    df_zod_annos = pd.DataFrame(zod_content)

    #make a dataframe with all results
    df_annos_full = pd.concat([df_avltruck_annos, df_avlrooftop_annos, df_zod_annos], axis=0) 
    df_annos_full = df_annos_full.reset_index(drop=True)

    #explode the lists
    cols_to_explode = ['names', 'gt_boxes_lidar', 'dataset', 'frame_id']
    df_annos_lists = [df_annos_full[col].explode() for col in cols_to_explode]
    df_annos_full = pd.concat(df_annos_lists, axis=1)

    prev_len = len(df_annos_full)
    df_annos_full.dropna(inplace=True)
    print(f"Dropped {prev_len - len(df_annos_full)} rows with NaNs")


    namemap_avltruck = {
        "Vehicle_Drivable_Car": "Vehicle",
        "Vehicle_Drivable_Van": "Vehicle",
        "Vehicle_Ridable_Motorcycle": "Cyclist",
        "Vehicle_Ridable_Bicycle": "Cyclist",
        "Human": "Pedestrian",
        "LargeVehicle_Bus": "Truck",
        "LargeVehicle_TruckCab": "Truck",
        "LargeVehicle_Truck": "Truck",
        "Trailer": "Truck",
        "Dont_Care": "DontCare",
        "Other": "DontCare",
        #'Placeholder': 'DontCare',
    }

    namemap_avlrooftop =  {
        "Vehicle_Drivable_Car": "Vehicle",
        "Vehicle_Drivable_Van": "Vehicle",
        "LargeVehicle_Truck": "Truck",
        "LargeVehicle_TruckCab": "Truck",
        "LargeVehicle_Bus": "Truck",
        "LargeVehicle_Bus_Bendy": "Truck",
        "Trailer": "Truck",
        "Vehicle_Ridable_Motorcycle": "Cyclist",
        "Vehicle_Ridable_Bicycle": "Cyclist",
        "Human": "Pedestrian",
        "PPObject": "DontCare",
        "PPObject_Stroller": "DontCare",
        "Dont_care": "DontCare",
        #'Placeholder': 'DontCare',
    }

    namemap_zod = {
        "Vehicle_Car": "Vehicle",
        "Vehicle_Van": "Vehicle",
        "Vehicle_Truck": "Truck",
        "Vehicle_Trailer": "Truck",
        "Vehicle_Bus": "Truck",
        "Vehicle_HeavyEquip": "Truck",
        "Vehicle_TramTrain": "Truck",
        "VulnerableVehicle_Motorcycle": "Cyclist",
        "VulnerableVehicle_Bicycle": "Cyclist",
        "Pedestrian": "Pedestrian",
        #'Placeholder': 'DontCare',
    }

    #apply name maps for all data of each dataset
    df_annos_full['names'] = df_annos_full['names'].apply(lambda x: namemap_avltruck[x] if x in namemap_avltruck.keys() else x)
    df_annos_full['names'] = df_annos_full['names'].apply(lambda x: namemap_avlrooftop[x] if x in namemap_avlrooftop.keys() else x)
    df_annos_full['names'] = df_annos_full['names'].apply(lambda x: namemap_zod[x] if x in namemap_zod.keys() else x)

    #drop all dont cares
    df_annos_full = df_annos_full[df_annos_full['names'] != 'DontCare']

    return df_annos_full

def store_detections_with_gt_boxes(detections, gt_labels, folder_name, split):
    #store the detections with the gt boxes
    detections_with_gt_boxes = detections.copy()
    detections_with_gt_boxes = detections_with_gt_boxes.merge(gt_labels[['gt_id', 'frame_id', 'gt_boxes_lidar']], on=['gt_id', 'frame_id'], how='left', suffixes=('', '_gt'))

    # dict for each frame with keys name, score, boxes_lidar, pred_labels, frame_id
    detections_with_gt_boxes_per_frame = {}
    for frame_id in detections_with_gt_boxes['frame_id'].unique():
        detections_with_gt_boxes_per_frame[frame_id] = {}

        detections_with_gt_boxes_frame = detections_with_gt_boxes[detections_with_gt_boxes['frame_id'] == frame_id]

        detections_with_gt_boxes_per_frame[frame_id]['name'] = detections_with_gt_boxes_frame['name'].values
        detections_with_gt_boxes_per_frame[frame_id]['score'] = detections_with_gt_boxes_frame['score'].values
        detections_with_gt_boxes_per_frame[frame_id]['boxes_lidar'] = detections_with_gt_boxes_frame['boxes_lidar'].values
        detections_with_gt_boxes_per_frame[frame_id]['pred_labels'] = detections_with_gt_boxes_frame['pred_labels'].values
        detections_with_gt_boxes_per_frame[frame_id]['frame_id'] = detections_with_gt_boxes_frame['frame_id'].values
        detections_with_gt_boxes_per_frame[frame_id]['gt_boxes_lidar'] = detections_with_gt_boxes_frame['gt_boxes_lidar'].values

    #store the detections with gt boxes
    with open(f"/home/cgriesbacher/thesis/3DTrans/gace/gace_output/{folder_name}/detections_with_gt_boxes_{split}.pkl", 'wb') as f:
        pkl.dump(detections_with_gt_boxes_per_frame, f)

In [ ]:
split = "val"
gt_labels_all = load_gt_labels(split)
gt_labels_all["dataset"].value_counts()
#add a gt id to each gt per frame
gt_labels_all['gt_id'] = gt_labels_all.groupby(['dataset', 'frame_id']).cumcount()


In [ ]:
folder_name = "2023-12-18_15-16-02"     #avltrck -> ZOD

source_dataset_name = "avltruck"
target_dataset_name = "zod"

results_path_avltruck = "/home/cgriesbacher/thesis/3DTrans/gace/gace_output/"+folder_name+"/det_annos.pkl"
detections_no_gace = pd.DataFrame()
detections_no_gace = load_detections(results_path_avltruck, detections_no_gace, train_dataset=source_dataset_name, eval_dataset=target_dataset_name)

#add a detection id to each detection per frame
detections_no_gace['det_id'] = detections_no_gace.groupby(['frame_id']).cumcount()

detections_all = detections_no_gace.copy()


In [36]:
#reduce gt labels to frames that are in the detections
gt_labels_all = gt_labels_all[gt_labels_all['frame_id'].isin(detections_all['frame_id'].unique())]

#add x, y, range, angle
detections_all['x'] = detections_all['boxes_lidar'].apply(lambda x: x[0])
detections_all['y'] = detections_all['boxes_lidar'].apply(lambda x: x[1])
detections_all['range'] = np.sqrt(detections_all['x']**2 + detections_all['y']**2)
detections_all['angle'] = np.arctan2(detections_all['y'], detections_all['x'])*180/np.pi

gt_labels_all['x'] = gt_labels_all['gt_boxes_lidar'].apply(lambda x: x[0])
gt_labels_all['y'] = gt_labels_all['gt_boxes_lidar'].apply(lambda x: x[1])
gt_labels_all['range'] = np.sqrt(gt_labels_all['x']**2 + gt_labels_all['y']**2)
gt_labels_all['angle'] = np.arctan2(gt_labels_all['y'], gt_labels_all['x'])*180/np.pi


#align datasets
min_angle = -60
max_angle = 60
max_range = 140

detections_all = detections_all[(detections_all['angle'] >= min_angle) & (detections_all['angle'] <= max_angle)]
detections_all = detections_all[detections_all['range'] < max_range]

gt_labels_all = gt_labels_all[(gt_labels_all['angle'] >= min_angle) & (gt_labels_all['angle'] <= max_angle)]
gt_labels_all = gt_labels_all[gt_labels_all['range'] < max_range]

#filter frames with no gts or detections
detections_all = detections_all[detections_all['frame_id'].isin(gt_labels_all['frame_id'].unique())]
gt_labels_all = gt_labels_all[gt_labels_all['frame_id'].isin(detections_all['frame_id'].unique())]

detections_all.reset_index(drop=True, inplace=True)
gt_labels_all.reset_index(drop=True, inplace=True)


#calc ious
detections_vehicles = detections_all[detections_all['name'] == 'Vehicle']
gt_labels_vehicles = gt_labels_all[gt_labels_all['names'] == 'Vehicle']
detections_vehicles, gt_labels_vehicles = calc_ious(detections_vehicles, gt_labels_vehicles)


detectoins_predestrians = detections_all[detections_all['name'] == 'Pedestrian']
gt_labels_predestrians = gt_labels_all[gt_labels_all['names'] == 'Pedestrian']
detections_pedestrians, gt_labels_pedestrians = calc_ious(detectoins_predestrians, gt_labels_predestrians)

detections_cyclists = detections_all[detections_all['name'] == 'Cyclist']
gt_labels_cyclists = gt_labels_all[gt_labels_all['names'] == 'Cyclist']
detections_cyclists, gt_labels_cyclists = calc_ious(detections_cyclists, gt_labels_cyclists)

detections = pd.DataFrame()
detections = pd.concat([detections_vehicles, detections_pedestrians, detections_cyclists], axis=0)


100%|██████████| 455/455 [00:00<00:00, 513.93it/s]


In [64]:
store_detections_with_gt_boxes(detections, gt_labels_all, folder_name, split)